In [12]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score, f1_score, roc_auc_score, log_loss
)
from xgboost import XGBClassifier
from INAFEN import InterpretableAutomatedFeatureEngineering

# Optional: H-measure (install via pip if you want it)
try:
    from hmeasure import h_score
    hmeasure_available = True
except ImportError:
    hmeasure_available = False
    

# Load dataset
data_df = pd.read_csv('datasets/pima indians diabetes.csv')

# Preprocessing
df_feas = data_df.iloc[:, :-1]
df_target = data_df.iloc[:, -1]

scaler = StandardScaler()
df_feas_scaled = pd.DataFrame(scaler.fit_transform(df_feas), columns=df_feas.columns)

label_encoder = LabelEncoder()
df_target_encoded = pd.Series(label_encoder.fit_transform(df_target), name='classes')

df_preprocessed = pd.concat([df_feas_scaled, df_target_encoded], axis=1)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    df_feas_scaled, df_target_encoded, test_size=0.2, random_state=42
)

# Train teacher model
teacher_model = XGBClassifier(n_estimators=100, max_depth=3, gamma=1, eval_metric='logloss')
teacher_model.fit(X_train, y_train)

# Prepare INAFEN
num_categorical_feas = 0
num_numerical_feas = X_train.shape[1]

INAFEN_model = InterpretableAutomatedFeatureEngineering(
    DTFT=True,
    ARFC=True,
    BMKD=True,
    number_of_categorical_features=num_categorical_feas,
    number_of_numerical_features=num_numerical_feas,
    FC_min_support=0.2,
    FC_min_confidence=0.85,
    ST_temperature=1,
    ST_alpha=1
)

# Fit INAFEN
INAFEN_model.fit(
    X=pd.DataFrame(X_train, columns=df_feas.columns),
    y_true=pd.DataFrame(y_train, columns=['classes']),
    y_soft=teacher_model.predict_proba(X_train)[:, 1]
)

# Predict & evaluate
X_test_transformed = INAFEN_model.transform(pd.DataFrame(X_test, columns=df_feas.columns))
y_pred = INAFEN_model.predict(X_test_transformed)
y_pred_proba = INAFEN_model.predict_proba(X_test_transformed)[:, 1]
# Metrics
acc = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_pred_proba)
lloss = log_loss(y_test, y_pred_proba)

if hmeasure_available:
    h = h_score(y_test.values, y_pred_proba)
else:
    h = 'Unavailable'

# Display results
print(f"\n Evaluation Metrics for INAFEN:")
print(f"Accuracy     : {acc:.4f}")
print(f"F1 Score     : {f1:.4f}")
print(f"AUC Score    : {auc:.4f}")
print(f"Log Loss     : {lloss:.4f}")
print(f"H-measure    : {h if h != 'Unavailable' else 'Install hmeasure for this'}")


XTEST
             6       148        72        35         0      33.6     0.627  \
667  0.002852 -0.283659 -0.213733  0.067348  0.157877 -0.005814 -0.107625   
324 -0.198030  0.068302 -0.096203 -0.193585  0.157877 -0.241838 -0.193148   
623 -0.198030 -0.172758 -0.213733 -0.060693 -0.072905 -0.005814 -0.193148   
689  0.189738 -0.172758 -0.004482 -0.060693 -0.072905 -0.302006  0.024376   
521  0.002852 -0.172758 -0.213733 -0.060693 -0.072905 -0.302006 -0.193148   
..        ...       ...       ...       ...       ...       ...       ...   
355 -0.198030  0.068302 -0.213733  0.067348  0.157877 -0.005814  0.246603   
533 -0.198030 -0.283659 -0.213733 -0.193585 -0.219067 -0.005814  0.246603   
344  0.189738  0.068302  0.178566  0.067348 -0.219067 -0.005814 -0.107625   
296 -0.198030  0.068302  0.178566 -0.193585  0.157877 -0.005814 -0.107625   
720 -0.198030 -0.172758 -0.096203  0.067348  0.157877 -0.005814 -0.107625   

           50  ['148-[[-inf, -0.6683599948883057]]']  
667  0.120258

In [5]:
import os
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, log_loss
from xgboost import XGBClassifier
from INAFEN import InterpretableAutomatedFeatureEngineering
import warnings

warnings.filterwarnings('ignore')

# --- Step 1: Load and preprocess raw data ---
dataset_path = 'datasets/heloc.csv'
data_df = pd.read_csv(dataset_path)

# Filter out rows with 20 or more -9s
data_df = data_df.loc[(data_df == -9).sum(axis=1) < 20].reset_index(drop=True)

# Replace coded missing values with np.nan
data_df = data_df.replace([-7, -8, -9], np.nan)

# Feature columns (excluding target)
feature_cols = data_df.columns[1:]
mean_imputing_cols = feature_cols.tolist()

# Mean imputation
mean_imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
data_df[mean_imputing_cols] = mean_imputer.fit_transform(data_df[mean_imputing_cols])

# Separate features and target
df_feas = data_df.iloc[:, 1:]
df_target = data_df.iloc[:, 0]

# Encode target labels
label_encoder = LabelEncoder()
df_target = pd.DataFrame(label_encoder.fit_transform(df_target), columns=['classes'])

# Combine into full dataframe
df_preprocessed = pd.concat([df_feas, df_target], axis=1)
X = df_feas.values
y = df_target.values.ravel()
df_preprocessed_columns = df_feas.columns

# --- Step 2: Prepare for INAFEN training ---
num_categorical_feas = 0
num_numerical_feas = len(df_feas.columns)

args = {
    'DTFT': True,
    'ARFC': True,
    'BMKD': True,
    'TBD_max_depth_dict': None,
    'FC_min_support': 0.2,
    'FC_min_confidence': 0.85,
    'ST_temperature': 1,
    'ST_alpha': 1
}

# --- Step 3: Train/Test split ---
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Train a teacher model for soft labels
teacher_model = XGBClassifier(n_estimators=100, max_depth=3, gamma=1, use_label_encoder=False, eval_metric='logloss')
teacher_model.fit(X_train, y_train)

# Initialize and train INAFEN
INAFEN_model = InterpretableAutomatedFeatureEngineering(
    DTFT=args['DTFT'],
    ARFC=args['ARFC'],
    BMKD=args['BMKD'],
    number_of_categorical_features=num_categorical_feas,
    number_of_numerical_features=num_numerical_feas,
    TBD_max_depth_dict=args['TBD_max_depth_dict'],
    FC_min_support=args['FC_min_support'],
    FC_min_confidence=args['FC_min_confidence'],
    ST_temperature=args['ST_temperature'],
    ST_alpha=args['ST_alpha']
)

INAFEN_model.fit(
    X=pd.DataFrame(X_train, columns=df_preprocessed_columns),
    y_true=pd.DataFrame(y_train, columns=['classes']),
    y_soft=teacher_model.predict_proba(X_train)[:, 1]
)

# --- Step 4: Evaluate ---
X_test_transformed = INAFEN_model.transform(pd.DataFrame(X_test, columns=df_preprocessed_columns))
y_pred_proba = INAFEN_model.predict_proba(X_test_transformed)[:, 1]
y_pred = INAFEN_model.predict(X_test_transformed)

# Accuracy
accuracy = accuracy_score(y_test, y_pred)

# AUC
auc = roc_auc_score(y_test, y_pred_proba)

# F1 Score
f1 = f1_score(y_test, y_pred)

# Log Loss
lloss = log_loss(y_test, y_pred_proba)

# H-measure (try using optional hmeasure module, otherwise skip)
try:
    from hmeasure import h_score
    h_measure = h_score(y_test, y_pred_proba)
except Exception as e:
    h_measure = f"H-measure error: {e}"

except ImportError:
    h_measure = "H-measure module not installed"

# --- Print all metrics ---
print(f"Final Accuracy:   {accuracy:.4f}")
print(f"Final AUC:        {auc:.4f}")
print(f"Final F1 Score:   {f1:.4f}")
print(f"Final Log Loss:   {lloss:.4f}")
print(f"Final H-measure:  {h_measure}")


Final Accuracy:   0.5205
Final AUC:        0.5000
Final F1 Score:   0.0000
Final Log Loss:   0.6925
Final H-measure:  0.0


In [9]:
import os
import time
import numpy as np
import pandas as pd
import warnings
from sklearn.metrics import f1_score, log_loss, roc_auc_score, accuracy_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier

from INAFEN import InterpretableAutomatedFeatureEngineering

# --- Ignore warnings
warnings.filterwarnings('ignore')

# --- Load data
data_path = 'datasets/'
dataset_name = 'bank marketing.csv'
data_df = pd.read_csv(os.path.join(data_path, dataset_name), delimiter=';')

# --- Clean column names (in case of trailing spaces)
data_df.columns = data_df.columns.str.strip()

# --- Data preprocessing
df_target = data_df['y']
df_feas = data_df.drop(columns=['y'])

categorical_cols = df_feas.select_dtypes(include=['object']).columns.tolist()
numerical_cols = df_feas.select_dtypes(include=['int64', 'float64']).columns.tolist()

num_categorical_feas = len(categorical_cols)
num_numerical_feas = len(numerical_cols)

df_categorical_encoded = pd.get_dummies(df_feas[categorical_cols], drop_first=True)

scaler = StandardScaler()
df_numerical_scaled = pd.DataFrame(scaler.fit_transform(df_feas[numerical_cols]), columns=numerical_cols)

df_feas_processed = pd.concat([df_numerical_scaled, df_categorical_encoded], axis=1)

label_encoder = LabelEncoder()
df_target_encoded = pd.DataFrame(label_encoder.fit_transform(df_target), columns=['classes'])

# --- Final dataset
X = df_feas_processed.values
y = df_target_encoded.values.ravel()
data_df_columns = df_feas_processed.columns

# --- Split once into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# --- Further split for teacher training and validation
X_train_train, X_train_valid, y_train_train, y_train_valid = train_test_split(X_train, y_train, test_size=0.2, stratify=y_train, random_state=42)

# --- Train teacher model
teacher_model = XGBClassifier(n_estimators=100, max_depth=3, gamma=1, random_state=42)
teacher_model.fit(X_train_train, y_train_train)

# --- Train INAFEN
parameter_dict = {
    'FC_min_support': 0.2,
    'FC_min_confidence': 0.85
}

model = InterpretableAutomatedFeatureEngineering(
    DTFT=True,
    ARFC=True,
    BMKD=True,
    number_of_categorical_features=num_categorical_feas,
    number_of_numerical_features=num_numerical_feas,
    TBD_max_depth_dict=None,
    **parameter_dict
)

model.fit(
    X=pd.DataFrame(X_train_train, columns=data_df_columns),
    y_true=pd.DataFrame(y_train_train, columns=['classes']),
    y_soft=teacher_model.predict_proba(X_train_train)[:, 1]
)

# --- Transform and Evaluate
X_val_trans = model.transform(pd.DataFrame(X_train_valid, columns=data_df_columns))
X_test_trans = model.transform(pd.DataFrame(X_test, columns=data_df_columns))

val_preds = model.predict_proba(X_val_trans)[:, 1]
test_preds = model.predict_proba(X_test_trans)[:, 1]

val_auc = roc_auc_score(y_train_valid, val_preds)
test_auc = roc_auc_score(y_test, test_preds)

val_f1 = f1_score(y_train_valid, np.round(val_preds))
test_f1 = f1_score(y_test, np.round(test_preds))

val_logloss = log_loss(y_train_valid, val_preds)
test_logloss = log_loss(y_test, test_preds)

val_acc = accuracy_score(y_train_valid, np.round(val_preds))
test_acc = accuracy_score(y_test, np.round(test_preds))

try:
    from hmeasure import h_score
    val_h = h_score(y_train_valid, val_preds)
    test_h = h_score(y_test, test_preds)
except ImportError:
    val_h = test_h = "hmeasure module not installed"

# --- Print results
print(f"Validation AUC:  {val_auc:.4f}, Accuracy: {val_acc:.4f}, F1: {val_f1:.4f}, LogLoss: {val_logloss:.4f}, H: {val_h}")
print(f"Test AUC:        {test_auc:.4f}, Accuracy: {test_acc:.4f}, F1: {test_f1:.4f}, LogLoss: {test_logloss:.4f}, H: {test_h}")


Validation AUC:  0.9003, Accuracy: 0.9011, F1: 0.3826, LogLoss: 0.2662, H: 0.5166960285428823
Test AUC:        0.9100, Accuracy: 0.9073, F1: 0.4203, LogLoss: 0.2481, H: 0.5529728350587362
